In [1]:
#from cuml.svm import SVC

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from joblib import Parallel, delayed
import itertools
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.base import clone

In [2]:
from train_test import load_data, filter_data, encode_labels
X, y, study_labels = load_data("/home/jeppe/Documents/Leukem.ai/data")
X, y, study_labels = filter_data(X, y, study_labels)
y, label_mapping = encode_labels(y)

  Studies: 2974
  X shape: (2974, 60660)
  y: 2974
  Studies: 1914
  X shape: (1914, 60660)
  y: 1914


In [7]:
import transformers

# CV setup
outer_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Hyperparameter grid
param_grid = {
    'n_genes': [1000, 2000],
    'C': [100, 1000],
    'gamma': ['auto'],
    'kernel': ['rbf'],
    'class_weight': ["balanced", None],
    'probability': [True]
}

param_combos = list(ParameterGrid(param_grid))

model = SVC
pipe = Pipeline([
    ('DEseq2', transformers.DESeq2RatioNormalizer()),
    ('feature_selection', transformers.FeatureSelection()),
    ('scaler', StandardScaler())
])

In [ ]:
# Function to evaluate one inner fold + hyperparam combo
def evaluate_inner_fold(outer_fold, inner_fold,
                        processed_X, y_train_inner, y_val_inner,
                        model,
                        params,
                        type = "standard"):
    
      
    def standard_eval():
        y_train_inner = np.array(y_train_inner, dtype=np.int32)
        y_val_inner = np.array(y_val_inner, dtype=np.int32)
        clf.fit(X_train_inner, y_train_inner)
        preds = clf.predict_proba(X_val_inner)
        preds = preds[:, 1]
        preds = (preds >= 0.5).astype(int)
        return {
            'outer_fold': outer_fold,
            'inner_fold': inner_fold,
            'params': params,
            'accuracy': accuracy_score(y_val_inner, preds),
            'f1_macro': f1_score(y_val_inner, preds, average='macro'),
            'f1_per_class': f1_score(y_val_inner, preds, average=None)
        }

    def ovr_eval():
        results = []
        classes = np.unique(y_train_inner)
        for cl in classes:
            y_train_bin = [1 if yy == cl else 0 for yy in y_train_inner]
            y_val_bin = [1 if yy == cl else 0 for yy in y_val_inner]

            y_train_bin = np.array(y_train_bin, dtype=np.int32)
            y_val_bin = np.array(y_val_bin, dtype=np.int32)

            clf.fit(X_train_inner, y_train_bin)
            preds = clf.predict_proba(X_val_inner)
            preds = preds[:, 1]
            preds = (preds >= 0.5).astype(int)
            results.append({
                'outer_fold': outer_fold,
                'inner_fold': inner_fold,
                'class': cl,
                'params': params,
                'accuracy': accuracy_score(y_val_bin, preds),
                'f1_binary': f1_score(y_val_bin, preds, average='binary', pos_label=1)
            })
        return results

    def ovo_eval():
        results = []
        classes = np.unique(y_train_inner)
        for i, j in itertools.combinations(classes, 2):
            train_mask = [(yy == i or yy == j) for yy in y_train_inner]
            val_mask = [(yy == i or yy == j) for yy in y_val_inner]

            X_train_ij = X_train_inner[train_mask]
            y_train_ij = np.array([yy for yy in y_train_inner if yy == i or yy == j], dtype=np.int32) 

            X_val_ij = X_val_inner[val_mask]
            y_val_ij = np.array([yy for yy in y_val_inner if yy == i or yy == j], dtype=np.int32)
            
            clf.fit(X_train_ij, y_train_ij)
            preds = clf.predict_proba(X_val_ij)
            preds = preds[:, 1]
            preds = (preds >= 0.5).astype(int)
            results.append({
                'outer_fold': outer_fold,
                'inner_fold': inner_fold,
                'class_0': i,
                'class_1': j,
                'params': params,
                'accuracy': accuracy_score(y_val_ij, preds),
                'f1_binary': f1_score(y_val_ij, preds, average='binary', pos_label=i)
            })
        return results

    # Dispatch table for clean logic
    eval_dispatch = {
        'standard': standard_eval,
        'OvR': ovr_eval,
        'OvO': ovo_eval
    }

    if type not in eval_dispatch:
        raise ValueError(f"Unsupported evaluation type: {type}")
    
    # Select preprocessed data
    n_genes = params.pop('n_genes')
    X_train_inner = processed_X[n_genes][0]
    X_val_inner = processed_X[n_genes][1]

    # Set classifier
    clf = clone(model(**params))
    params['n_genes'] = n_genes

    return eval_dispatch[type]()

def pre_process_data(n_genes_list, X_train_outer, train_inner_idx, val_inner_idx, study_labels_outer, pipe):
        
        X_train_inner = X_train_outer[train_inner_idx]
        X_val_inner = X_train_outer[val_inner_idx]

        study_labels_inner = study_labels_outer[train_inner_idx]
        
        y_train_inner = y_train_outer[train_inner_idx]
        y_val_inner = y_train_outer[val_inner_idx]

        processed_X = {}
        for n_genes_i in n_genes_list:
            pipe_inner = clone(pipe)

            X_train_inner_proc = pipe_inner.fit_transform(X_train_inner, 
                                                feature_selection__study_per_patient=study_labels_inner, 
                                                feature_selection__n_genes=n_genes_i)
            X_val_inner_proc = pipe_inner.transform(X_val_inner)


            processed_X[n_genes_i] = [X_train_inner_proc, X_val_inner_proc]
        return processed_X, y_train_inner, y_val_inner

all_results = []

X= np.array(X, dtype=np.float32)

for outer_fold, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
    X_train_outer, X_test_outer = X[train_idx], X[test_idx]
    y_train_outer, y_test_outer = y[train_idx], y[test_idx]
    study_labels_outer = study_labels[train_idx]
    
    inner_tasks = []
    for inner_fold, (train_inner_idx, val_inner_idx) in enumerate(inner_cv.split(X_train_outer, y_train_outer)):

        processed_X, y_train_inner, y_val_inner = pre_process_data(
            param_grid["n_genes"], 
            X_train_outer, 
            train_inner_idx, 
            val_inner_idx, 
            study_labels_outer,
            pipe)

        for params in param_combos:
            inner_tasks.append(delayed(evaluate_inner_fold)(
                outer_fold, inner_fold,
                processed_X, y_train_inner, y_val_inner,
                model,
                params,
                type = "OvR" # standard, OvR, OvO
            ))

    # Run inner CV tasks in parallel (adjust n_jobs to number of CPU cores)
    inner_results = Parallel(n_jobs=8, verbose=1)(inner_tasks)
    if isinstance(inner_results[0], dict):
        # Flat list of dictionaries
        all_results.extend(inner_results)
    elif isinstance(inner_results[0], list):
        # List of lists of dictionaries
        for res in inner_results:
            all_results.extend(res)
    else:
        raise ValueError("Unexpected structure in inner_results")


# Convert to DataFrame
df_parallel_results = pd.DataFrame(all_results)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 out of  16 | elapsed:   29.5s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 out of  16 | elapsed:   30.0s finished


In [19]:
def process_cv_results(df, param_grid, label_mapping, score_col='f1_binary'):
    #  Extract param names and expand 'params'
    param_names = list(param_grid.keys())
    params_df = df['params'].apply(pd.Series)

    # Normalize None values for groupby
    for col in param_names:
        if col in params_df.columns:
            params_df[col] = params_df[col].apply(lambda x: 'none' if x is None else x)

    #Combine expanded params with original DataFrame
    df_with_params = pd.concat([df.drop(columns=['params']), params_df], axis=1)

    # Determine group-by strategy based on evaluation type
    if 'class' in df_with_params.columns:
        # OvR
        group_cols = param_names + ['class']
        summary = df_with_params.groupby(group_cols)[score_col].mean().reset_index()
        best = summary.loc[summary.groupby('class')[score_col].idxmax()].reset_index(drop=True)

    elif 'class_0' in df_with_params.columns and 'class_1' in df_with_params.columns:
        # OvO
        group_cols = param_names + ['class_0', 'class_1']
        summary = df_with_params.groupby(group_cols)[score_col].mean().reset_index()
        best = summary.loc[summary.groupby(['class_0', 'class_1'])[score_col].idxmax()].reset_index(drop=True)

    else:
        # Standard multiclass
        group_cols = param_names
        score_col = 'f1_macro'  # override if not passed
        summary = df_with_params.groupby(group_cols)[score_col].mean().reset_index()
        best = summary.loc[[summary[score_col].idxmax()]].reset_index(drop=True)

    int_to_label = {v: k for k, v in label_mapping.items()}
    if 'class' in best.columns:
        # OvR case
        best['class_label'] = best['class'].map(int_to_label)
        return best

    elif 'class_0' in best.columns and 'class_1' in best.columns:
        # OvO case
        best['class_0_label'] = best['class_0'].map(int_to_label)
        best['class_1_label'] = best['class_1'].map(int_to_label)
        return best

    else:
        return best

best_per_class_df = process_cv_results(
    df_parallel_results,
    param_grid=param_grid,
    label_mapping = label_mapping
)
best_per_class_df

,n_genes,C,gamma,kernel,class_weight,probability,class,f1_binary,class_label
0,2000,100,auto,rbf,none,True,0,0.558774,AML with MDS-related cytogenetic abnormalities
1,2000,1000,auto,rbf,balanced,True,1,0.696370,AML with MDS-related gene mutations
2,1000,100,auto,rbf,balanced,True,2,0.799741,AML with in-frame bZIP CEBPA
3,1000,100,auto,rbf,balanced,True,3,0.982587,AML with inv(16)/t(16;16)/CBFB::MYH11
4,2000,1000,auto,rbf,balanced,True,4,0.655357,AML with inv(3)/t(3;3)/GATA2;MECOM
5,2000,1000,auto,rbf,none,True,5,0.961694,AML with mutated NPM1
6,2000,1000,auto,rbf,none,True,6,0.597064,AML with mutated TP53
7,2000,1000,auto,rbf,none,True,7,0.897777,AML with t(6;9)/DEK::NUP214
8,2000,1000,auto,rbf,none,True,8,0.966720,AML with t(8;21)/RUNX1::RUNX1T1
9,2000,1000,auto,rbf,none,True,9,0.723048,AML with t(9;11)/MLLT3::KMT2A


In [ ]:
type_eval = "OvR"
columns_to_drop = ["f1_binary", "class_label", "class_0_label", "class_1_label"]
columns_to_drop_existing = [col for col in columns_to_drop if col in best_per_class_df.columns]

per_class_results = []
overall_results = []

for outer_fold, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
    
    df = best_per_class_df

    X_train_outer, X_test_outer = X[train_idx], X[test_idx]
    y_train_outer, y_test_outer = y[train_idx], y[test_idx]
    study_labels_outer = study_labels[train_idx]

    classes_in_fold = np.unique(y_train_outer)
    
    def pre_process_data(df):
        processed_X = {}
        for n_genes_i in list(set(df["n_genes"])):
            pipe_outer = clone(pipe)    
            X_train_n_genes = pipe_outer.fit_transform(X_train_outer, 
                                                    feature_selection__study_per_patient=study_labels_outer, 
                                                    feature_selection__n_genes=n_genes_i)
            X_test_n_genes = pipe_outer.transform(X_test_outer)
            processed_X[n_genes_i] = [X_train_n_genes, X_test_n_genes]
        return processed_X
    
    processed_X = pre_process_data(df)
    results_per_class = []

    def ovr_eval():
        df = df.drop(columns=columns_to_drop_existing)
        df = df[df['class'].isin(classes_in_fold)]

        prob_df = pd.DataFrame(index = test_idx, columns = classes_in_fold)

        for cl in classes_in_fold:        
            params = df[df["class"] == cl]
            params = params.iloc[0].to_dict()

            n_genes = params.pop('n_genes')
            params.pop('class', None)

            X_train_n_genes = processed_X[n_genes][0]
            X_test_n_genes = processed_X[n_genes][1]

            if params["class_weight"]=='none':
                params.pop('class_weight')
                
            clf = clone(model(**params))
            params['n_genes'] = n_genes

            y_train_bin = [1 if yy == cl else 0 for yy in y_train_outer]
            y_test_bin = [1 if yy == cl else 0 for yy in y_test_outer]

            y_train_bin = np.array(y_train_bin, dtype=np.int32)
            y_test_bin = np.array(y_test_bin, dtype=np.int32)

            clf.fit(X_train_n_genes, y_train_bin)

            preds_proba = clf.predict_proba(X_test_n_genes)[:, 1]
            preds = (preds_proba >= 0.5).astype(int)
            results_per_class.append({
                'outer_fold': outer_fold,
                'class': cl,
                'accuracy': accuracy_score(y_test_bin, preds),
                'f1_binary': f1_score(y_test_bin, preds, average='binary', pos_label=1)
            })
            prob_df[cl] = preds_proba

        results_overall = {
            'outer_fold': outer_fold,
            'accuracy': accuracy_score(y_test_outer, prob_df.idxmax(axis=1)),
            'f1_macro': f1_score(y_test_outer, prob_df.idxmax(axis=1), average='macro')
        }
        return results_overall, results_per_class

    def ovo_eval():
        df = df[(df['class_0'].isin(classes_in_fold)) & (df['class_1'].isin(classes_in_fold))]

        results_per_class = []
        
        prob_df = pd.DataFrame(0, index = test_idx, columns = classes_in_fold)

        for index, df_row in df.iterrows():
            i = df_row["class_0"]
            j = df_row["class_1"]

            params = df_row.to_dict()

            n_genes = params.pop('n_genes')
            params.pop('class_0', None)
            params.pop('class_1', None)

            X_train_n_genes = processed_X[n_genes][0]
            X_test_n_genes = processed_X[n_genes][1]

            train_mask = [(yy == i or yy == j) for yy in y_train_outer]
            val_mask = [(yy == i or yy == j) for yy in y_test_outer]

            X_train_ij = X_train_n_genes[train_mask]
            y_train_ij = (y_train_outer == i).astype(np.int32)

            X_val_ij = X_test_n_genes[val_mask]
            y_val_ij = (y_test_outer == i).astype(np.int32)
            
            clf = clone(model(**params))
            params['n_genes'] = n_genes
            
            clf.fit(X_train_ij, y_train_ij)
            preds = clf.predict(X_val_ij)
            results_per_class.append({
                'outer_fold': outer_fold,
                'class_0': i,
                'class_1': j,
                'accuracy': accuracy_score(y_val_ij, preds),
                'f1_binary': f1_score(y_val_ij, preds, average='binary', pos_label=1)
            })
            for prediction in preds:
                if prediction == 1:
                    prob_df[i] = prob_df[i] + 1
                else:
                    prob_df[j] = prob_df[j] + 1

        results_overall = {
            'outer_fold': outer_fold,
            'accuracy': accuracy_score(y_test_outer, prob_df.idxmax(axis=1)),
            'f1_macro': f1_score(y_test_outer, prob_df.idxmax(axis=1), average='macro'),
            'f1_binary': f1_score(y_test_outer, preds, prob_df.idxmax(axis=1), average=None)
        }
        return results_overall, results_per_class

    # Dispatch table for clean logic
    eval_dispatch = {
        'OvR': ovr_eval,
        'OvO': ovo_eval
    }

    if type_eval not in eval_dispatch:
        raise ValueError(f"Unsupported evaluation type: {type}")
    
    results_overall, results_per_class = eval_dispatch[type_eval]()
    per_class_results.append(results_per_class)
    overall_results.append(results_overall)

overall_results

[{'outer_fold': 0,
  'accuracy': 0.8505747126436781,
  'f1_macro': 0.7820579583287295},
 {'outer_fold': 1,
  'accuracy': 0.8537095088819227,
  'f1_macro': 0.7791393959181361}]